# 📊 AdventureWorks Sales Performance Analysis (SQL + BigQuery)

---

## 🧰 Tools Used
- Google BigQuery
- SQL (CTEs, JOINs, Aggregation, Filtering)
- Jupyter Notebook to document SQL queries, analysis walkthrough and business insights

---

## 🎯 Objective
This project analyzes sales and product data from the [AdventureWorks Dataset](https://www.kaggle.com/datasets/ukveteran/adventure-works?select=AdventureWorks_Products.csv) to uncover performance trends, high-value products and regional opportunities. The analysis is conducted using SQL on BigQuery and structured as a business-oriented case study.

---

## 📽 Pre-Processing

### Column Titles Cleaning
> _Decapitalization of Column Titles and Underscoring for consistency_
```sql
-- Example on Calendar Table
CREATE TABLE `AdventureWorks.calendar` AS
SELECT Date AS date
FROM `AdventureWorks.raw_calendar`
```

### Combining Yearly Sales Tables
> _Create a View of ALL Sales_
```sql
-- Union Sales per Year tables
CREATE VIEW `AdventureWorks.sales` AS
SELECT * FROM `AdventureWorks.sales_2015`
UNION ALL
SELECT * FROM `AdventureWorks.sales_2016`
UNION ALL
SELECT * FROM `AdventureWorks.sales_2017`
```

---

## 📁 Dataset Overview
The following tables are used:
- `products`, `categories`, `subcategories`: product details and pricing
- `sales`: order data
- `sales_2015`, `sales_2016`, `sales_2017`: order data per year
- `customers`: customer info
- `territories`: sales region

---

## 1. Schema Exploration
> _List available tables and row counts_  
> _Use COUNT(*) and metadata queries to understand table sizes and contents_
```sql
-- Count of rows per table
SELECT 'products' AS table_name, COUNT(*) AS row_count
FROM `AdventureWorks.products`
UNION ALL
SELECT 'sales' AS table_name, COUNT(*) AS row_count
FROM `AdventureWorks.sales`
UNION ALL
SELECT 'customers' AS table_name, COUNT(*) AS row_count
FROM `AdventureWorks.customers`
``` 

## 2. Total Sales Overview
> _Create a View of Orders with Price_  
> _Orders Cost, Price & Revenue_
```sql
-- Cost, Price & Revenue by Order
CREATE VIEW `AdventureWorks.orders` AS
SELECT
    s.order_date,
    s.order_number,
    s.territory_key,
    s.product_key,
    s.order_quantity,
    p.product,
    ROUND(p.cost, 2) AS product_cost,
    ROUND(p.price, 2) AS product_price,
    ROUND((s.order_quantity * p.cost), 2) AS order_cost,
    ROUND((s.order_quantity * p.price), 2) AS order_price,
    ROUND((s.order_quantity * p.price) - (s.order_quantity * p.cost), 2) AS order_revenue
FROM `AdventureWorks.sales` s
JOIN `AdventureWorks.products` p
    ON s.product_key = p.product_key
```

## 3. Exploratory Business Questions
> _Exploration on Products, Sales, Revenue and Customers_

- #### 📃 Top Sales by Product
```sql
-- Top 10 Product Sales and Quantities Sold 
SELECT
    product,
    ROUND(SUM(order_price), 2) AS total_sales,
    SUM(order_quantity) AS quantity_sales
FROM `AdventureWorks.orders`
GROUP BY product
ORDER BY total_sales DESC
LIMIT 10
```
> ##### _"Mountain and road bikes dominate sales volumes, likely due to their higher popularity and broader market"_

- #### 📈 Best Selling Categories by Revenue
```sql
-- Categories by Revenue and Quantities Sold 
SELECT
    c.category AS category,
    ROUND(SUM(o.order_revenue), 2) AS total_revenue,
    SUM(o.order_quantity) AS quantity_sales
FROM `AdventureWorks.orders` o
JOIN `AdventureWorks.products` p
    ON o.product_key = p.product_key
JOIN `AdventureWorks.subcategories` sc
    ON p.subcategory_key = sc.subcategory_key
JOIN `AdventureWorks.categories` c
    ON sc.category_key = c.category_key
GROUP BY category
ORDER BY total_revenue DESC
```
> ##### _"Bikes are the highest-revenue category, however Accessories dominate on quantities sold."_

- #### 📊 Top Selling Sub-Categories by Revenue
```sql
-- Top 10 Sub-Categories by Revenue and Quantities Sold 
SELECT
    sc.subcategory AS subcategory,
    ROUND(SUM(o.order_revenue), 2) AS total_revenue,
    SUM(o.order_quantity) AS quantity_sales
FROM `AdventureWorks.orders` o
JOIN `AdventureWorks.products` p
    ON o.product_key = p.product_key
JOIN `AdventureWorks.subcategories` sc
    ON p.subcategory_key = sc.subcategory_key
GROUP BY subcategory
ORDER BY total_revenue DESC
LIMIT 10
```
> ##### _"Again we see Road Bikes as the most profitable Category, while Tires and Tubes have the most quantities sold. Sugesting targeted promotions can be very profitable in these Sub-Categories."_

- #### 🛍 Top Selling Products by Revenue
```sql
-- Top 10 Product Revenue and Quantities Sold 
SELECT
    product,
    ROUND(SUM(order_revenue), 2) AS total_revenue,
    SUM(order_quantity) AS quantity_sales
FROM `AdventureWorks.orders`
GROUP BY product
ORDER BY total_revenue DESC
LIMIT 10
```
> ##### _"The Top 5 Products contribute to a large amount of the total product revenue. Counting over 50% of total product revenue."_


- #### 📅 Sales Over Time (Month, Quarter, Year)
> Sales by Year
```sql
-- Overview on Sales by Year
SELECT
    EXTRACT(YEAR FROM order_date) AS year,
    SUM(order_quantity) AS sales_quantity,
    ROUND(SUM(order_cost), 2) AS total_sales_cost,
    ROUND(SUM(order_price), 2) AS total_sales,
    ROUND(SUM(order_revenue), 2) AS total_revenue
FROM `AdventureWorks.orders`
GROUP BY year
ORDER BY year ASC
```

> Sales by Quarter
```sql
-- Overview on Sales by Quarter
SELECT
    CONCAT('Q', EXTRACT(QUARTER FROM order_date)) AS quarter,
    EXTRACT(YEAR FROM order_date) AS year,
    SUM(order_quantity) AS sales_quantity,
    ROUND(SUM(order_cost), 2) AS total_sales_cost,
    ROUND(SUM(order_price), 2) AS total_sales,
    ROUND(SUM(order_revenue), 2) AS total_revenue
FROM `AdventureWorks.orders`
GROUP BY
    year,
    quarter
ORDER BY
    year,
    quarter
```

> Sales by Month
```sql
-- Overview on Sales by Month
SELECT
    FORMAT_DATE('%m-%Y', order_date) AS month,
    SUM(order_quantity) AS sales_quantity,
    ROUND(SUM(order_cost), 2) AS total_sales_cost,
    ROUND(SUM(order_price), 2) AS total_sales,
    ROUND(SUM(order_revenue), 2) AS total_revenue
FROM `AdventureWorks.orders`
GROUP BY month
ORDER BY PARSE_DATE('%m-%Y', month) ASC
```
> ##### _"Sales by Year show higher demand throughout the years."_
> ##### _"Sales show steady growth month-over-month, suggesting minimal seasonal fluctuation."_

- #### 🌍 Sales by Region
```sql
-- Overview on Sales by Region
SELECT
    t.country AS country,
    t.region AS region,
    COUNT(o.order_number) AS order_count,
    SUM(o.order_quantity) AS sales_quantity,
    ROUND(SUM(o.order_cost), 2) AS total_sales_cost,
    ROUND(SUM(o.order_price), 2) AS total_sales,
    ROUND(SUM(o.order_revenue), 2) AS total_revenue
FROM `AdventureWorks.orders` o
JOIN `AdventureWorks.territories` t
    ON o.territory_key = t.territory_key
GROUP BY
    t.country,
    t.region
ORDER BY total_sales DESC
```
> ##### _"North America generates the most revenue, particularly on the West side. But emerging sales in Europe and Australia indicate a potential area for expansion."_

- #### 👤 Most Frequent Customers
```sql
-- Top 10 Most Frequent Customers
-- Orders Sales and Revenue by Customer
SELECT
    CONCAT(c.first_name, ' ', c.last_name) AS customer_name,
    s.customer_key AS customer_key,
    COUNT(o.order_number) AS order_count,
    SUM(o.order_quantity) AS sales_quantity,
    ROUND(SUM(o.order_price), 2) AS total_sales,
    ROUND(SUM(o.order_revenue), 2) AS total_revenue
FROM `AdventureWorks.sales` s
JOIN `AdventureWorks.customers` c
    ON s.customer_key = c.customer_key
JOIN `AdventureWorks.orders` o
    ON s.order_number = o.order_number
GROUP BY
    s.customer_key,
    customer_name
ORDER BY total_orders DESC
```
> ##### _"High-revenue customers with high-quantity orders indicate ideal opportunities for loyalty programs."_
  
---

## 🧠 Insights & Recommendations
- **Focus on High-Performing Products**  
  ➤ A small group of products (mainly bikes and accessories) drive most revenue.  
  ➤ Prioritize inventory planning, marketing and promotions around these.  
- **Understand Growth, Not Seasonality**  
  ➤ Sales are increasing consistently month-over-month, with no clear seasonal dips or spikes.  
  ➤ Keep monitoring trends to identify potential cycles.  
- **Target Growth regions**  
  ➤ North America is the core market, but Europe and Australia show strong potential.  
  ➤ These should be prioritized for expansion or targeted campaigns.  
- **Leverage Customer Loyalty**  
  ➤ Top customers make frequent, high-value purchases.  
  ➤ Implement segmentation and loyalty programs to nurture these realationships.

---

## 📌 What's Next?
- 📍 Segment customers by geography or product preferences to deepen insights
- 📍 Incorporate product return/cancellation data for net revenue accuracy
- 📍 Visualize trends using Python (matplotlib, seaborn) or Power BI
- 📍 Export key tables to CSV for sharing and reporting